# 深入MNIST

这里，我们使用更加方便的InteractiveSession类。通过它，你可以更加灵活地构建你的代码。它能让你在运行图的时候，插入一些计算图，这些计算图是由某些操作(operations)构成的。这对于使用IPython、jupyter note等的人们来说非常便利。如果你没有使用InteractiveSession，那么你需要在启动session之前构建整个计算图，然后启动该计算图。

## 为什么使用计算图，而不是直接计算

为了在Python中进行高效的数值计算，我们通常会使用像NumPy一类的库，将一些诸如矩阵乘法的耗时操作在Python环境的外部来计算，这些计算通常会通过其它语言并用更为高效的代码来实现。

但遗憾的是，每一个操作切换回Python环境时仍需要不小的开销。如果你想在GPU或者分布式环境中计算时，这一开销更加可怖，这一开销主要可能是用来进行数据迁移。

因此Python代码的目的是用来构建这个可以在外部运行的计算图，以及安排计算图的哪一部分应该被运行。

In [1]:
import tensorflow as tf
import py_code.input_data as input_data

In [3]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
sess = tf.InteractiveSession()

In [5]:
# 占位符
x = tf.placeholder(float, [None, 784])
y_ = tf.placeholder(float, [None, 10])

In [6]:
# 变量
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

In [8]:
sess.run(tf.global_variables_initializer())

In [9]:
# 类别预测
y = tf.nn.softmax(tf.matmul(x, W) + b)

In [10]:
# 损失函数
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

In [11]:
# 训练模型
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

In [12]:
for i in range(1000):
  batch = mnist.train.next_batch(50)
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})

In [13]:
# 评估模型
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

In [14]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [15]:
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.909


可以看到和上一节差不多的步骤得到了，和上一节差不多的结果

## 构建一个多层卷积网络

### 权重初始化

这里我们加入一些噪声，用以打破对称性，以及避免0梯度

In [16]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)